In [1]:
import datetime

from influxdb_client import InfluxDBClient, Point, Dialect
from influxdb_client.client.write_api import SYNCHRONOUS

In [2]:
host = "http://localhost:8086"
token = "mB_Kr-Ak8SwqPByhbH25ozucJGE5DUAzOF1zLRLoJ39ndpBa-0NAaE6FWp7AHeY6323P2Gv2ESi28IycphHXZg=="
org = "iciclespy3"
bucket = "iciclespy3"

In [3]:
client = InfluxDBClient(url=host, token=token, org=org)
query_api = client.query_api()

In [10]:
p = {"_start": datetime.timedelta(hours=-1),
     '_end': datetime.datetime.now(),
     '_my_measurement': 'Temperature',
     '_min_t': 0,
     '_max_t': 20}

tables = query_api.query(f'from(bucket:"{bucket}") |> range(start: _start, stop: _end) |> filter(fn: (r) => r._measurement == _my_measurement and _min_t < r._value and r._value < _max_t)', params=p)

In [11]:
for table in tables:
    print(table)
    for record in table.records:
        print(record.values)

FluxTable() columns: 15, records: 1
{'result': '_result', 'table': 0, '_start': datetime.datetime(2021, 4, 30, 17, 14, 30, 445690, tzinfo=tzutc()), '_stop': datetime.datetime(2021, 4, 30, 21, 14, 30, 444685, tzinfo=tzutc()), '_time': datetime.datetime(2021, 4, 30, 18, 2, 54, 861075, tzinfo=tzutc()), '_value': 19.480563403927626, 'Building': '25-27', 'City': 'Moscow', 'Country': 'Russia', 'Device_token': 'DEV1', 'Index': '1', 'Region_state': 'Moscow', 'Street': 'Taganskaya', '_field': 'C', '_measurement': 'Temperature'}
